# Evaluation
## Setup

In [ ]:
import analysis.aggregator as ag
import data.file_handler as fh
from analysis.pipeline_blocks import calc_ari, calc_ari_per_token, add_sense_counts_to_id_map
from data.corpus_handler import CorpusName, CorpusHandler
import plotly.express as px

In [ ]:
LABEL_DICT = {'unique_sense_count': 'Unique Sense Count',
              'total_token_count': 'Total Token Count',
              'unique_token_count': 'Unique Token Count',
              'ari': 'ARI'}
WIDTH = 512
HEIGHT = 256

## Corpus Evaluation

In [ ]:
corpus = CorpusHandler(CorpusName.SEMCOR, './data/corpus_cache')
st = ag.calc_corpus_statistics_for_tagged_senses(corpus)
st

In [ ]:
tagged_tokens = corpus.get_tagged_tokens()

### Complete Disambiguation Baseline: One cluster/sense per token

In [ ]:
tagged_tokens['sense'] = range(tagged_tokens.token.count())
calc_ari(corpus.get_tagged_tokens(), tagged_tokens)

### No Disambiguation Baseline: One cluster/sense per unique token

In [ ]:
tagged_tokens['sense'] = tagged_tokens.token.factorize()[0]
calc_ari(corpus.get_tagged_tokens(), tagged_tokens)

### Relationship Between Sense Counts and Token Counts

In [ ]:
tagged_tokens = corpus.get_tagged_tokens()
tagged_tokens_and_counts = ag.count_unique_senses_per_token(tagged_tokens[tagged_tokens.tagged_sense])
token_counts_per_sense_count = ag.count_tokens_per_sense_count(tagged_tokens_and_counts)
token_counts_per_sense_count.head()

In [ ]:
fig = px.scatter(token_counts_per_sense_count, x='unique_sense_count', y='unique_token_count',
                 color='total_token_count', size='total_token_count', color_continuous_scale=px.colors.sequential.Greys,
                 log_x=True, log_y=True, labels=LABEL_DICT,
                 template='plotly_white', width=WIDTH, height=WIDTH)
fig.update_layout(coloraxis_colorbar=dict(title_side='right',
                                          thicknessmode="fraction", thickness=0.04,
                                          ticks="outside"))
fig.update_traces(marker={'line': {'color': 'black'}})

fig.show()
#fig.write_image(f'data/plots/fig_{corpus.corpus_name}_tokens_and_senses.pdf')

In [ ]:
tagged_tokens_and_counts[tagged_tokens_and_counts.unique_sense_count > 50].sort_values(by='unique_sense_count')

## Dictionary Evaluation

In [ ]:
ABS_PATH = fh.add_and_get_abs_path('./data/results')
#dictionary = fh.load_df(ABS_PATH, 'toy-affinity_cosine-linkage_single-dist_0.4-dictionary.pkl')
#dictionary = fh.load_df(ABS_PATH, 'semcor-affinity_cosine-linkage_single-dist_0.4-dictionary.pkl')

In [ ]:
ag.pack_sentence_ids_and_token_ids(
    ag.unpack_and_sort_per_token_id(
        dictionary,
        ['sentence_id', 'token_id', 'sense']),
    ['token', 'sense'])

In [ ]:
dictionary = calc_ari_per_token(corpus.get_tagged_tokens(), dictionary)
fig_ari_per_token = px.histogram(dictionary[dictionary.tagged_token], x='ari', range_x=[-1, 1],
                                 labels=LABEL_DICT,
                                 template='plotly_white', width=WIDTH, height=HEIGHT)
fig_ari_per_token.show()

In [ ]:
if 'unique_sense_count' not in dictionary.columns:
    dictionary = add_sense_counts_to_id_map(corpus.get_tagged_tokens(), dictionary)
ari_per_sense_count = dictionary[dictionary.tagged_token].groupby(by='unique_sense_count').aggregate({'ari': 'mean'}).reset_index()
ari_per_sense_count

In [ ]:
fig_ari_per_sense_count = px.bar(ari_per_sense_count, x='unique_sense_count', y='ari',
                                 labels=LABEL_DICT, template='plotly_white', width=WIDTH, height=HEIGHT)
fig_ari_per_sense_count.update_xaxes(
    showgrid=False, ticks="outside", tickson="labels", ticklen=4,
    nticks=int(ari_per_sense_count.unique_sense_count.max()))

fig_ari_per_sense_count.show()